In [ ]:
# Check installed versions
import pandas as pd
import numpy as np
import sklearn
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1


In [ ]:
# Import required modules
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1


In [ ]:
# Define possible values with expanded cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data
np.random.seed(42)
n_samples = 1000
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(lambda x: min(len(set(x) & set(key_pages)), 3))
df['preferences_specified'] = (df['foodPreferences'].apply(len) +
                               df['transportType'].apply(len) +
                               df['accommodationType'].apply(len))

# Define scoring functions (unchanged from original)
def target_city_score(x):
    return 15 if x != 'Not specified' else 0

def start_date_score(x):
    if x == 'Within 30 days':
        return 25
    elif x == '31-90 days':
        return 15
    elif x == 'More than 90 days':
        return 5
    else:
        return 0

def budget_score(x):
    if x == 'High':
        return 15
    elif x == 'Medium':
        return 10
    elif x == 'Low':
        return 5
    else:
        return 0

def engagement_score(pages, preferences):
    pages_score = min(pages * 3, 9)
    preferences_score = min(preferences * 1, 5)
    return pages_score + preferences_score

def contact_score(x):
    return 5 if x == 'Yes' else 0

def distance_score(x):
    if x == 'Long':
        return 10
    elif x == 'Medium':
        return 5
    elif x == 'Short':
        return 2
    else:
        return 0

def safety_score(x):
    if x == 'High':
        return 10
    elif x == 'Medium':
        return 5
    else:
        return 0

def income_score(x):
    if x == 'High':
        return 5
    elif x == 'Medium':
        return 3
    elif x == 'Low':
        return 1
    else:
        return 0

def lifestyle_score(x):
    if x == 'Luxury':
        return 5
    elif x == 'Active':
        return 3
    elif x == 'Relaxed':
        return 2
    elif x == 'Budget':
        return 1

# Apply scoring
df['target_city_score'] = df['targetCity'].apply(target_city_score)
df['start_date_score'] = df['startDate'].apply(start_date_score)
df['budget_score'] = df['budget'].apply(budget_score)
df['engagement_score'] = df.apply(lambda row: engagement_score(row['pages_visited'], row['preferences_specified']), axis=1)
df['contact_score'] = df['phone_provided'].apply(contact_score)
df['distance_score'] = df['distance'].apply(distance_score)
df['safety_score'] = df['safety'].apply(safety_score)
df['income_score'] = df['income'].apply(income_score)
df['lifestyle_score'] = df['lifestyle'].apply(lifestyle_score)

df['total_score'] = df[['target_city_score', 'start_date_score', 'budget_score', 'engagement_score',
                        'contact_score', 'distance_score', 'safety_score', 'income_score', 'lifestyle_score']].sum(axis=1)

In [ ]:
# Prepare data for machine learning
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']
X = df[categorical_features + numerical_features]
y = df['total_score']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validation R-squared Scores: {cv_scores}")
print(f"Mean CV R-squared: {np.mean(cv_scores):.2f}")

model.fit(X_train, y_train)

Cross-Validation R-squared Scores: [0.85877313 0.89215336 0.86587257 0.8892396  0.87907982]
Mean CV R-squared: 0.88


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['targetCity', 'startDate',
                                                   'budget', 'phone_provided',
                                                   'distance', 'safety',
                                                   'income', 'lifestyle']),
                                                 ('num', 'passthrough',
                                                  ['pages_visited',
                                                   'preferences_specified'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [ ]:
# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 22.71
R-squared: 0.89


In [ ]:
# Show sample predictions
print("\nSample Predictions:")
for actual, predicted in list(zip(y_test, y_pred))[:5]:
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")


Sample Predictions:
Actual Score: 55, Predicted Score: 50.67
Actual Score: 39, Predicted Score: 45.70
Actual Score: 82, Predicted Score: 78.49
Actual Score: 25, Predicted Score: 31.44
Actual Score: 76, Predicted Score: 75.39


**Method 2**

In [ ]:
# Check installed versions
import pandas as pd
import numpy as np
import sklearn
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

# Install xgboost and joblib if not present
try:
    import xgboost as xgb
    import joblib
except ImportError:
    !pip install xgboost joblib
    import xgboost as xgb
    import joblib

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define possible values with expanded Indian cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data (larger dataset)
np.random.seed(42)
n_samples = 10000  # Increased from 1000 to 10000
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(lambda x: min(len(set(x) & set(key_pages)), 3))
df['preferences_specified'] = (df['foodPreferences'].apply(len) +
                               df['transportType'].apply(len) +
                               df['accommodationType'].apply(len))

# Define scoring functions
def target_city_score(x):
    return 15 if x != 'Not specified' else 0

def start_date_score(x):
    if x == 'Within 30 days':
        return 25
    elif x == '31-90 days':
        return 15
    elif x == 'More than 90 days':
        return 5
    else:
        return 0

def budget_score(x):
    if x == 'High':
        return 15
    elif x == 'Medium':
        return 10
    elif x == 'Low':
        return 5
    else:
        return 0

def engagement_score(pages, preferences):
    pages_score = min(pages * 3, 9)
    preferences_score = min(preferences * 1, 5)
    return pages_score + preferences_score

def contact_score(x):
    return 5 if x == 'Yes' else 0

def distance_score(x):
    if x == 'Long':
        return 10
    elif x == 'Medium':
        return 5
    elif x == 'Short':
        return 2
    else:
        return 0

def safety_score(x):
    if x == 'High':
        return 10
    elif x == 'Medium':
        return 5
    else:
        return 0

def income_score(x):
    if x == 'High':
        return 5
    elif x == 'Medium':
        return 3
    elif x == 'Low':
        return 1
    else:
        return 0

def lifestyle_score(x):
    if x == 'Luxury':
        return 5
    elif x == 'Active':
        return 3
    elif x == 'Relaxed':
        return 2
    elif x == 'Budget':
        return 1

# Apply scoring
df['target_city_score'] = df['targetCity'].apply(target_city_score)
df['start_date_score'] = df['startDate'].apply(start_date_score)
df['budget_score'] = df['budget'].apply(budget_score)
df['engagement_score'] = df.apply(lambda row: engagement_score(row['pages_visited'], row['preferences_specified']), axis=1)
df['contact_score'] = df['phone_provided'].apply(contact_score)
df['distance_score'] = df['distance'].apply(distance_score)
df['safety_score'] = df['safety'].apply(safety_score)
df['income_score'] = df['income'].apply(income_score)
df['lifestyle_score'] = df['lifestyle'].apply(lifestyle_score)

df['total_score'] = df[['target_city_score', 'start_date_score', 'budget_score', 'engagement_score',
                        'contact_score', 'distance_score', 'safety_score', 'income_score', 'lifestyle_score']].sum(axis=1)

# Prepare data for machine learning
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']
X = df[categorical_features + numerical_features]
y = df['total_score']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model with XGBoost
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
print(f"Cross-Validation R-squared Scores: {cv_scores}")
print(f"Mean CV R-squared: {np.mean(cv_scores):.2f}")

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# Show sample predictions
print("\nSample Predictions:")
for actual, predicted in list(zip(y_test[:5], y_pred[:5])):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

# Save the model
joblib.dump(model, 'lead_scoring_model.pkl')
print("Model saved as 'lead_scoring_model.pkl'")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1
Cross-Validation R-squared Scores: [0.99588376 0.99527973 0.99578726 0.99586928 0.99557757]
Mean CV R-squared: 1.00
Mean Squared Error: 0.60
R-squared: 1.00

Sample Predictions:
Actual Score: 41, Predicted Score: 41.62
Actual Score: 74, Predicted Score: 74.73
Actual Score: 49, Predicted Score: 49.71
Actual Score: 49, Predicted Score: 48.88
Actual Score: 58, Predicted Score: 57.62
Model saved as 'lead_scoring_model.pkl'


In [ ]:
import pandas as pd
import numpy as np
import joblib

# Load the saved model
model = joblib.load('lead_scoring_model.pkl')
print("Model loaded successfully")

# Define possible values (must match training script)
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Example input data to match your sample predictions
sample_data = [
    {'targetCity': 'Bangalore', 'startDate': 'Within 30 days', 'budget': 'Medium', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'High', 'income': 'Medium', 'lifestyle': 'Active',
     'pages_visited': 2, 'preferences_specified': 3},  # Should predict ~55
    {'targetCity': 'Mumbai', 'startDate': '31-90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Medium', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 1, 'preferences_specified': 2},  # Should predict ~39
    {'targetCity': 'New Delhi', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Long', 'safety': 'High', 'income': 'High', 'lifestyle': 'Luxury',
     'pages_visited': 3, 'preferences_specified': 5},  # Should predict ~82
    {'targetCity': 'Not specified', 'startDate': 'More than 90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Low', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 0, 'preferences_specified': 1},  # Should predict ~25
    {'targetCity': 'Hyderabad', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'Medium', 'income': 'Medium', 'lifestyle': 'Relaxed',
     'pages_visited': 3, 'preferences_specified': 4},  # Should predict ~76
]

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# Predict scores
predictions = model.predict(sample_df)

# Display results
print("\nSample Predictions:")
for actual, predicted in zip([55, 39, 82, 25, 76], predictions):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

Model loaded successfully

Sample Predictions:
Actual Score: 55, Predicted Score: 83.56
Actual Score: 39, Predicted Score: 49.69
Actual Score: 82, Predicted Score: 100.74
Actual Score: 25, Predicted Score: 19.59
Actual Score: 76, Predicted Score: 87.98


**Method 3**

In [ ]:
# Check installed versions
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import pickle
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"TensorFlow version: {tf.__version__}")

# Import required modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Define possible values with expanded Indian cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data (larger dataset)
np.random.seed(42)
n_samples = 10000  # 10,000 samples
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(lambda x: min(len(set(x) & set(key_pages)), 3))
df['preferences_specified'] = (df['foodPreferences'].apply(len) +
                               df['transportType'].apply(len) +
                               df['accommodationType'].apply(len))

# Define scoring functions
def target_city_score(x):
    return 15 if x != 'Not specified' else 0

def start_date_score(x):
    if x == 'Within 30 days':
        return 25
    elif x == '31-90 days':
        return 15
    elif x == 'More than 90 days':
        return 5
    else:
        return 0

def budget_score(x):
    if x == 'High':
        return 15
    elif x == 'Medium':
        return 10
    elif x == 'Low':
        return 5
    else:
        return 0

def engagement_score(pages, preferences):
    pages_score = min(pages * 3, 9)
    preferences_score = min(preferences * 1, 5)
    return pages_score + preferences_score

def contact_score(x):
    return 5 if x == 'Yes' else 0

def distance_score(x):
    if x == 'Long':
        return 10
    elif x == 'Medium':
        return 5
    elif x == 'Short':
        return 2
    else:
        return 0

def safety_score(x):
    if x == 'High':
        return 10
    elif x == 'Medium':
        return 5
    else:
        return 0

def income_score(x):
    if x == 'High':
        return 5
    elif x == 'Medium':
        return 3
    elif x == 'Low':
        return 1
    else:
        return 0

def lifestyle_score(x):
    if x == 'Luxury':
        return 5
    elif x == 'Active':
        return 3
    elif x == 'Relaxed':
        return 2
    elif x == 'Budget':
        return 1

# Apply scoring
df['target_city_score'] = df['targetCity'].apply(target_city_score)
df['start_date_score'] = df['startDate'].apply(start_date_score)
df['budget_score'] = df['budget'].apply(budget_score)
df['engagement_score'] = df.apply(lambda row: engagement_score(row['pages_visited'], row['preferences_specified']), axis=1)
df['contact_score'] = df['phone_provided'].apply(contact_score)
df['distance_score'] = df['distance'].apply(distance_score)
df['safety_score'] = df['safety'].apply(safety_score)
df['income_score'] = df['income'].apply(income_score)
df['lifestyle_score'] = df['lifestyle'].apply(lifestyle_score)

df['total_score'] = df[['target_city_score', 'start_date_score', 'budget_score', 'engagement_score',
                        'contact_score', 'distance_score', 'safety_score', 'income_score', 'lifestyle_score']].sum(axis=1)

# Prepare data for machine learning
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']
X = df[categorical_features + numerical_features]
y = df['total_score']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Build a deeper DNN model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),  # Increased neurons
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model with a lower learning rate for finer tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])

# Train the model with more epochs
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# Show sample predictions
print("\nSample Predictions:")
for actual, predicted in list(zip(y_test[:5], y_pred[:5])):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

# Save the model in native Keras format and preprocessor with pickle
model.save('lead_scoring_dnn.keras')
with open('preprocessor_config.pkl', 'wb') as f:
    pickle.dump(preprocessor.named_transformers_['cat'].categories_, f)
print("Model saved as 'lead_scoring_dnn.keras' and preprocessor config saved as 'preprocessor_config.pkl'")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1
TensorFlow version: 2.18.0
Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1738.3478 - mae: 33.4532 - val_loss: 5.8040 - val_mae: 1.6873
Epoch 2/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 16.6755 - mae: 3.1810 - val_loss: 2.6587 - val_mae: 1.3472
Epoch 3/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 13.1532 - mae: 2.8567 - val_loss: 2.2344 - val_mae: 1.2547
Epoch 4/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.6757 - mae: 2.6912 - val_loss: 0.7586 - val_mae: 0.6598
Epoch 5/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.3349 - mae: 2.6234 - val_loss: 1.0532 - val_mae: 0.8218
Epoch 6/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.2504 - mae: 2.6035 - val_loss: 1.1907 - val_mae: 0.9083
Epoch 7/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.0500 - mae: 2.6149 - val_loss: 0.8193 - val_mae: 0.7222
Epoch 8/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 9.8547 - mae: 2.4408 - val_loss: 4.6993 - val_mae: 2.0138
Epoch 9/150
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/st

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the saved model
model = tf.keras.models.load_model('lead_scoring_dnn.keras')
print("Model loaded successfully")

# Define possible values (must match training script)
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Load preprocessor configuration
with open('preprocessor_config.pkl', 'rb') as f:
    categories = pickle.load(f)
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']

# Reconstruct the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(categories=categories, handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Example input data to match your sample predictions
sample_data = [
    {'targetCity': 'Bangalore', 'startDate': 'Within 30 days', 'budget': 'Medium', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'High', 'income': 'Medium', 'lifestyle': 'Active',
     'pages_visited': 2, 'preferences_specified': 3},  # Should predict ~55
    {'targetCity': 'Mumbai', 'startDate': '31-90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Medium', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 1, 'preferences_specified': 2},  # Should predict ~39
    {'targetCity': 'New Delhi', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Long', 'safety': 'High', 'income': 'High', 'lifestyle': 'Luxury',
     'pages_visited': 3, 'preferences_specified': 5},  # Should predict ~82
    {'targetCity': 'Not specified', 'startDate': 'More than 90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Low', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 0, 'preferences_specified': 1},  # Should predict ~25
    {'targetCity': 'Hyderabad', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'Medium', 'income': 'Medium', 'lifestyle': 'Relaxed',
     'pages_visited': 3, 'preferences_specified': 4},  # Should predict ~76
]

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# Preprocess the sample data
sample_processed = preprocessor.fit_transform(sample_df)

# Predict scores
predictions = model.predict(sample_processed).flatten()

# Display results
print("\nSample Predictions:")
for actual, predicted in zip([55, 39, 82, 25, 76], predictions):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

Model loaded successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step

Sample Predictions:
Actual Score: 55, Predicted Score: 75.36
Actual Score: 39, Predicted Score: 46.72
Actual Score: 82, Predicted Score: 89.67
Actual Score: 25, Predicted Score: 18.12
Actual Score: 76, Predicted Score: 77.09


**Method 4**

In [ ]:
# Check installed versions
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import pickle
import xgboost as xgb
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"TensorFlow version: {tf.__version__}")

# Import required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, LayerNormalization, MultiHeadAttention, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Define possible values with expanded Indian cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data (larger dataset)
np.random.seed(42)
n_samples = 20000  # Increased to 20,000 samples
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(lambda x: min(len(set(x) & set(key_pages)), 3))
df['preferences_specified'] = (df['foodPreferences'].apply(len) +
                               df['transportType'].apply(len) +
                               df['accommodationType'].apply(len))

# Define scoring functions
def target_city_score(x):
    return 15 if x != 'Not specified' else 0

def start_date_score(x):
    if x == 'Within 30 days':
        return 25
    elif x == '31-90 days':
        return 15
    elif x == 'More than 90 days':
        return 5
    else:
        return 0

def budget_score(x):
    if x == 'High':
        return 15
    elif x == 'Medium':
        return 10
    elif x == 'Low':
        return 5
    else:
        return 0

def engagement_score(pages, preferences):
    pages_score = min(pages * 3, 9)
    preferences_score = min(preferences * 1, 5)
    return pages_score + preferences_score

def contact_score(x):
    return 5 if x == 'Yes' else 0

def distance_score(x):
    if x == 'Long':
        return 10
    elif x == 'Medium':
        return 5
    elif x == 'Short':
        return 2
    else:
        return 0

def safety_score(x):
    if x == 'High':
        return 10
    elif x == 'Medium':
        return 5
    else:
        return 0

def income_score(x):
    if x == 'High':
        return 5
    elif x == 'Medium':
        return 3
    elif x == 'Low':
        return 1
    else:
        return 0

def lifestyle_score(x):
    if x == 'Luxury':
        return 5
    elif x == 'Active':
        return 3
    elif x == 'Relaxed':
        return 2
    elif x == 'Budget':
        return 1

# Apply scoring
df['target_city_score'] = df['targetCity'].apply(target_city_score)
df['start_date_score'] = df['startDate'].apply(start_date_score)
df['budget_score'] = df['budget'].apply(budget_score)
df['engagement_score'] = df.apply(lambda row: engagement_score(row['pages_visited'], row['preferences_specified']), axis=1)
df['contact_score'] = df['phone_provided'].apply(contact_score)
df['distance_score'] = df['distance'].apply(distance_score)
df['safety_score'] = df['safety'].apply(safety_score)
df['income_score'] = df['income'].apply(income_score)
df['lifestyle_score'] = df['lifestyle'].apply(lifestyle_score)

df['total_score'] = df[['target_city_score', 'start_date_score', 'budget_score', 'engagement_score',
                        'contact_score', 'distance_score', 'safety_score', 'income_score', 'lifestyle_score']].sum(axis=1)

# Prepare data for machine learning
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']
X = df[categorical_features + numerical_features]
y = df['total_score']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Reshape for CNN and Transformer (add a time-like dimension)
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the hybrid model (CNN + Transformer + DNN)
inputs = Input(shape=(X_train_reshaped.shape[1], 1))
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = LayerNormalization()(x)
x = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)  # Transformer layer
x = LayerNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(1)(x)

hybrid_model = Model(inputs, outputs)
hybrid_model.compile(optimizer=Adam(learning_rate=0.0003), loss='mse', metrics=['mae'])

# Train the hybrid model
history = hybrid_model.fit(X_train_reshaped, y_train, epochs=200, batch_size=64, validation_split=0.2, verbose=1)

# Get predictions from the hybrid model
y_pred_hybrid = hybrid_model.predict(X_test_reshaped).flatten()

# Train an XGBoost model for ensembling
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Ensemble: Average predictions from hybrid and XGBoost
y_pred_ensemble = (y_pred_hybrid + y_pred_xgb) / 2

# Evaluate the ensemble model
mse = mean_squared_error(y_test, y_pred_ensemble)
r2 = r2_score(y_test, y_pred_ensemble)

print(f"Mean Squared Error (Ensemble): {mse:.2f}")
print(f"R-squared (Ensemble): {r2:.2f}")

# Show sample predictions
print("\nSample Predictions (Ensemble):")
for actual, predicted in list(zip(y_test[:5], y_pred_ensemble[:5])):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

# Save the models and preprocessor
hybrid_model.save('lead_scoring_hybrid.keras')
with open('preprocessor_config.pkl', 'wb') as f:
    pickle.dump(preprocessor.named_transformers_['cat'].categories_, f)
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
print("Hybrid model saved as 'lead_scoring_hybrid.keras', XGBoost model saved as 'xgb_model.pkl', and preprocessor config saved as 'preprocessor_config.pkl'")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1
TensorFlow version: 2.18.0
Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 618.4660 - mae: 18.7912 - val_loss: 39.6490 - val_mae: 5.7939
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 43.4121 - mae: 5.1361 - val_loss: 66.4050 - val_mae: 7.7601
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 30.8048 - mae: 4.3478 - val_loss: 151.6818 - val_mae: 11.9450
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 25.8035 - mae: 3.9755 - val_loss: 249.2889 - val_mae: 15.3260
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 23.7392 - mae: 3.8053 - val_loss: 292.0809 - val_mae: 16.6354
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 22.7178 - mae: 3.7205 - val_loss: 360.6365 - val_mae: 18.4866
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 21.3043 - mae: 3.6113 - val_loss: 371.4743 - val_mae: 18.7503
Epoch 8/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the saved models
hybrid_model = tf.keras.models.load_model('lead_scoring_hybrid.keras')
with open('xgb_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
print("Models loaded successfully")

# Define possible values (must match training script)
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Load preprocessor configuration
with open('preprocessor_config.pkl', 'rb') as f:
    categories = pickle.load(f)
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']

# Reconstruct the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(categories=categories, handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Example input data to match your sample predictions
sample_data = [
    {'targetCity': 'Bangalore', 'startDate': 'Within 30 days', 'budget': 'Medium', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'High', 'income': 'Medium', 'lifestyle': 'Active',
     'pages_visited': 2, 'preferences_specified': 3},  # Should predict ~55
    {'targetCity': 'Mumbai', 'startDate': '31-90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Medium', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 1, 'preferences_specified': 2},  # Should predict ~39
    {'targetCity': 'New Delhi', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Long', 'safety': 'High', 'income': 'High', 'lifestyle': 'Luxury',
     'pages_visited': 3, 'preferences_specified': 5},  # Should predict ~82
    {'targetCity': 'Not specified', 'startDate': 'More than 90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Low', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 0, 'preferences_specified': 1},  # Should predict ~25
    {'targetCity': 'Hyderabad', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'Medium', 'income': 'Medium', 'lifestyle': 'Relaxed',
     'pages_visited': 3, 'preferences_specified': 4},  # Should predict ~76
]

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# Preprocess the sample data
sample_processed = preprocessor.fit_transform(sample_df)
sample_processed_reshaped = sample_processed.reshape((sample_processed.shape[0], sample_processed.shape[1], 1))

# Predict scores with hybrid model
y_pred_hybrid = hybrid_model.predict(sample_processed_reshaped).flatten()

# Predict scores with XGBoost model
y_pred_xgb = xgb_model.predict(sample_processed)

# Ensemble predictions
y_pred_ensemble = (y_pred_hybrid + y_pred_xgb) / 2

# Display results
print("\nSample Predictions (Ensemble):")
for actual, predicted in zip([55, 39, 82, 25, 76], y_pred_ensemble):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

Models loaded successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Sample Predictions (Ensemble):
Actual Score: 55, Predicted Score: 71.53
Actual Score: 39, Predicted Score: 42.61
Actual Score: 82, Predicted Score: 86.76
Actual Score: 25, Predicted Score: 16.44
Actual Score: 76, Predicted Score: 73.57


**Method 5**

In [ ]:
# # Install required libraries if not present
# !pip install transformers torch xgboost

# Check installed versions
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pickle
import xgboost as xgb
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Import required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, LayerNormalization, MultiHeadAttention, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Define possible values with expanded Indian cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data
np.random.seed(42)
n_samples = 50000  # 50,000 samples
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(lambda x: min(len(set(x) & set(key_pages)), 3))
df['preferences_specified'] = (df['foodPreferences'].apply(len) +
                               df['transportType'].apply(len) +
                               df['accommodationType'].apply(len))

# Define scoring functions for ground truth
def target_city_score(x): return 15 if x != 'Not specified' else 0
def start_date_score(x): return {'Within 30 days': 25, '31-90 days': 15, 'More than 90 days': 5}.get(x, 0)
def budget_score(x): return {'High': 15, 'Medium': 10, 'Low': 5}.get(x, 0)
def engagement_score(pages, prefs): return min(pages * 3, 9) + min(prefs * 1, 5)
def contact_score(x): return 5 if x == 'Yes' else 0
def distance_score(x): return {'Long': 10, 'Medium': 5, 'Short': 2}.get(x, 0)
def safety_score(x): return {'High': 10, 'Medium': 5}.get(x, 0)
def income_score(x): return {'High': 5, 'Medium': 3, 'Low': 1}.get(x, 0)
def lifestyle_score(x): return {'Luxury': 5, 'Active': 3, 'Relaxed': 2, 'Budget': 1}.get(x, 0)

# Apply scoring
df['total_score'] = (df['targetCity'].apply(target_city_score) +
                     df['startDate'].apply(start_date_score) +
                     df['budget'].apply(budget_score) +
                     df.apply(lambda row: engagement_score(row['pages_visited'], row['preferences_specified']), axis=1) +
                     df['phone_provided'].apply(contact_score) +
                     df['distance'].apply(distance_score) +
                     df['safety'].apply(safety_score) +
                     df['income'].apply(income_score) +
                     df['lifestyle'].apply(lifestyle_score))

# Prepare data for hybrid model
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']
X = df[categorical_features + numerical_features]
y = df['total_score']

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Reshape for CNN and Transformer
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build and train the hybrid neural network
inputs = Input(shape=(X_train_reshaped.shape[1], 1))
x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(inputs)
x = LayerNormalization()(x)
x = MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
x = LayerNormalization()(x)
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1)(x)

hybrid_model = Model(inputs, outputs)
hybrid_model.compile(optimizer=Adam(learning_rate=0.0002), loss='mse', metrics=['mae'])
history = hybrid_model.fit(X_train_reshaped, y_train, epochs=100, batch_size=128, validation_split=0.2, verbose=1)

# Get hybrid predictions
y_pred_hybrid = hybrid_model.predict(X_test_reshaped).flatten()

# Train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=300, learning_rate=0.03, max_depth=7, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Prepare data for DistilBERT (textual input)
def format_text(row):
    return (f"Target City: {row['targetCity']}, Start Date: {row['startDate']}, Budget: {row['budget']}, "
            f"Phone Provided: {row['phone_provided']}, Distance: {row['distance']}, Safety: {row['safety']}, "
            f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, Pages Visited: {row['pages_visited']}, "
            f"Preferences Specified: {row['preferences_specified']}")

df['text'] = df[categorical_features + numerical_features].apply(format_text, axis=1)
train_texts = df.iloc[y_train.index]['text'].tolist()
test_texts = df.iloc[y_test.index]['text'].tolist()
train_labels = y_train.tolist()
test_labels = y_test.tolist()

# Tokenize data for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Custom Dataset class for PyTorch
class LeadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LeadDataset(train_encodings, train_labels)
test_dataset = LeadDataset(test_encodings, test_labels)

# Fine-tune DistilBERT
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# Get DistilBERT predictions
predictions = trainer.predict(test_dataset)
y_pred_distilbert = predictions.predictions.flatten()

# Ensemble: Combine Hybrid, XGBoost, and DistilBERT predictions
y_pred_ensemble = (y_pred_hybrid * 0.4 + y_pred_xgb * 0.3 + y_pred_distilbert * 0.3)

# Evaluate the ensemble model
mse = mean_squared_error(y_test, y_pred_ensemble)
r2 = r2_score(y_test, y_pred_ensemble)

print(f"Mean Squared Error (Ensemble): {mse:.2f}")
print(f"R-squared (Ensemble): {r2:.2f}")

# Show sample predictions
print("\nSample Predictions (Ensemble):")
for actual, predicted in list(zip(y_test[:5], y_pred_ensemble[:5])):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

# Save models and preprocessor
hybrid_model.save('lead_scoring_hybrid.keras')
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
model.save_pretrained('distilbert_lead_scoring')
tokenizer.save_pretrained('distilbert_lead_scoring')
with open('preprocessor_config.pkl', 'wb') as f:
    pickle.dump(preprocessor.named_transformers_['cat'].categories_, f)
print("Models saved: 'lead_scoring_hybrid.keras', 'xgb_model.pkl', 'distilbert_lead_scoring', and 'preprocessor_config.pkl'")

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1
TensorFlow version: 2.18.0
PyTorch version: 2.6.0+cu124
Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - loss: 487.3631 - mae: 15.9710 - val_loss: 87.5744 - val_mae: 9.0452
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 19.2688 - mae: 3.4448 - val_loss: 120.0648 - val_mae: 10.6000
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 15.0761 - mae: 3.0416 - val_loss: 202.1741 - val_mae: 13.9102
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 12.5909 - mae: 2.7711 - val_loss: 276.7074 - val_mae: 16.2607
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 11.1614 - mae: 2.6200 - val_loss: 333.1827 - val_mae: 17.7917
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 10.2384 - mae: 2.5074 - val_loss: 344.4908 - val_mae: 18.0872
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 9.9267 - mae: 2.4732 - val_loss: 419.8454 - val_mae: 20.0368
Epoch

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pjdevelop (pjdevelop-Lovely Professional University) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,3.333300,2.790906
2,3.957600,0.368029
3,2.904100,0.174801


Mean Squared Error (Ensemble): 52.02
R-squared (Ensemble): 0.72

Sample Predictions (Ensemble):
Actual Score: 39, Predicted Score: 34.34
Actual Score: 48, Predicted Score: 42.05
Actual Score: 50, Predicted Score: 43.61
Actual Score: 64, Predicted Score: 56.00
Actual Score: 59, Predicted Score: 51.75
Models saved: 'lead_scoring_hybrid.keras', 'xgb_model.pkl', 'distilbert_lead_scoring', and 'preprocessor_config.pkl'


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pickle
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the saved models
hybrid_model = tf.keras.models.load_model('lead_scoring_hybrid.keras')
with open('xgb_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert_lead_scoring')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert_lead_scoring')
print("Models loaded successfully")

# Define possible values
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Load preprocessor configuration
with open('preprocessor_config.pkl', 'rb') as f:
    categories = pickle.load(f)
categorical_features = ['targetCity', 'startDate', 'budget', 'phone_provided', 'distance', 'safety', 'income', 'lifestyle']
numerical_features = ['pages_visited', 'preferences_specified']

# Reconstruct the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(categories=categories, handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Example input data
sample_data = [
    {'targetCity': 'Bangalore', 'startDate': 'Within 30 days', 'budget': 'Medium', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'High', 'income': 'Medium', 'lifestyle': 'Active',
     'pages_visited': 2, 'preferences_specified': 3},  # ~55
    {'targetCity': 'Mumbai', 'startDate': '31-90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Medium', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 1, 'preferences_specified': 2},  # ~39
    {'targetCity': 'New Delhi', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Long', 'safety': 'High', 'income': 'High', 'lifestyle': 'Luxury',
     'pages_visited': 3, 'preferences_specified': 5},  # ~82
    {'targetCity': 'Not specified', 'startDate': 'More than 90 days', 'budget': 'Low', 'phone_provided': 'No',
     'distance': 'Short', 'safety': 'Low', 'income': 'Low', 'lifestyle': 'Budget',
     'pages_visited': 0, 'preferences_specified': 1},  # ~25
    {'targetCity': 'Hyderabad', 'startDate': 'Within 30 days', 'budget': 'High', 'phone_provided': 'Yes',
     'distance': 'Medium', 'safety': 'Medium', 'income': 'Medium', 'lifestyle': 'Relaxed',
     'pages_visited': 3, 'preferences_specified': 4},  # ~76
]

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# Preprocess for hybrid and XGBoost
sample_processed = preprocessor.fit_transform(sample_df)
sample_processed_reshaped = sample_processed.reshape((sample_processed.shape[0], sample_processed.shape[1], 1))

# Predict with hybrid model
y_pred_hybrid = hybrid_model.predict(sample_processed_reshaped).flatten()

# Predict with XGBoost model
y_pred_xgb = xgb_model.predict(sample_processed)

# Prepare text for DistilBERT
def format_text(row):
    return (f"Target City: {row['targetCity']}, Start Date: {row['startDate']}, Budget: {row['budget']}, "
            f"Phone Provided: {row['phone_provided']}, Distance: {row['distance']}, Safety: {row['safety']}, "
            f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, Pages Visited: {row['pages_visited']}, "
            f"Preferences Specified: {row['preferences_specified']}")

sample_texts = sample_df.apply(format_text, axis=1).tolist()
sample_encodings = tokenizer(sample_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Predict with DistilBERT
distilbert_model.eval()
with torch.no_grad():
    outputs = distilbert_model(**sample_encodings)
    y_pred_distilbert = outputs.logits.squeeze().numpy()

# Ensemble predictions
y_pred_ensemble = (y_pred_hybrid * 0.4 + y_pred_xgb * 0.3 + y_pred_distilbert * 0.3)

# Display results
print("\nSample Predictions (Ensemble):")
for actual, predicted in zip([55, 39, 82, 25, 76], y_pred_ensemble):
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}")

Models loaded successfully
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Sample Predictions (Ensemble):
Actual Score: 55, Predicted Score: 74.35
Actual Score: 39, Predicted Score: 43.34
Actual Score: 82, Predicted Score: 88.43
Actual Score: 25, Predicted Score: 13.23
Actual Score: 76, Predicted Score: 77.01


**Method 5 Retry**

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.8 MB/s eta 0:00:00


In [3]:
# # Install required libraries if not present
# !pip install transformers torch xgboost scikit-learn==1.3.0 optuna

# Import required modules
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import pickle
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, LayerNormalization, MultiHeadAttention, Flatten, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import seaborn as sns

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")

# Enable mixed precision training for faster execution
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define possible values with expanded Indian cities
possible_cities = [
    'Bangalore', 'New Delhi', 'Mumbai', 'Kolkata', 'Chennai', 'Hyderabad',
    'Pune', 'Ahmedabad', 'Jaipur', 'Lucknow', 'Surat', 'Kanpur', 'Nagpur',
    'Patna', 'Bhopal', 'Indore', 'Vadodara', 'Coimbatore', 'Visakhapatnam',
    'Guwahati', 'Thiruvananthapuram', 'Kochi', 'Mysore', 'Goa', 'Chandigarh',
    'Amritsar', 'Jodhpur', 'Udaipur', 'Agra', 'Varanasi', 'Dehradun',
    'Ranchi', 'Jamshedpur', 'Bhubaneswar', 'Raipur', 'Not specified'
]
possible_start_dates = ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified']
possible_durations = ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified']
possible_budgets = ['High', 'Medium', 'Low', 'Not specified']
possible_incomes = ['High', 'Medium', 'Low', 'Not specified']
possible_lifestyles = ['Active', 'Relaxed', 'Luxury', 'Budget']
possible_distances = ['Long', 'Medium', 'Short', 'Not specified']
possible_safeties = ['High', 'Medium', 'Low', 'Not specified']
possible_phone = ['Yes', 'No']
possible_pages = ['home', 'about', 'services', 'pricing', 'contact', 'blog']
key_pages = ['services', 'pricing', 'contact']
possible_food = ['Vegetarian', 'Vegan', 'Gluten-free', 'None']
possible_transport = ['Car', 'Public Transit', 'Walking', 'Biking']
possible_accommodation = ['Hotel', 'Apartment', 'House', 'Hostel']

# Generate synthetic data with more samples
np.random.seed(42)
n_samples = 100000  # Increased from 50,000 to 100,000 samples
data = {
    'email': ['email@example.com'] * n_samples,
    'phone_provided': np.random.choice(possible_phone, n_samples),
    'currentCity': np.random.choice(possible_cities, n_samples),
    'targetCity': np.random.choice(possible_cities, n_samples),
    'startDate': np.random.choice(possible_start_dates, n_samples),
    'duration': np.random.choice(possible_durations, n_samples),
    'budget': np.random.choice(possible_budgets, n_samples),
    'income': np.random.choice(possible_incomes, n_samples),
    'lifestyle': np.random.choice(possible_lifestyles, n_samples),
    'distance': np.random.choice(possible_distances, n_samples),
    'safety': np.random.choice(possible_safeties, n_samples),
    'pagesVisited': [list(np.random.choice(possible_pages, np.random.randint(0, 7), replace=False)) for _ in range(n_samples)],
    'foodPreferences': [list(np.random.choice(possible_food, np.random.randint(0, 4), replace=False)) for _ in range(n_samples)],
    'transportType': [list(np.random.choice(possible_transport, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
    'accommodationType': [list(np.random.choice(possible_accommodation, np.random.randint(0, 5), replace=False)) for _ in range(n_samples)],
}
df = pd.DataFrame(data)

# Set phone based on phone_provided
df['phone'] = df['phone_provided'].apply(lambda x: '1234567890' if x == 'Yes' else '')

# Function to check if key pages were visited
def key_pages_visited(pages_list):
    return sum(1 for page in pages_list if page in key_pages)

# Compute numerical features
df['pages_visited'] = df['pagesVisited'].apply(len)
df['key_pages_visited'] = df['pagesVisited'].apply(key_pages_visited)
df['food_preferences'] = df['foodPreferences'].apply(len)
df['transport_preferences'] = df['transportType'].apply(len)
df['accommodation_preferences'] = df['accommodationType'].apply(len)
df['preferences_specified'] = df['food_preferences'] + df['transport_preferences'] + df['accommodation_preferences']

# Create interaction features
df['key_pages_ratio'] = df['key_pages_visited'] / df['pages_visited'].clip(lower=1)
df['budget_income_match'] = (df['budget'] == df['income']).astype(int)
df['is_local_travel'] = ((df['currentCity'] != 'Not specified') &
                         (df['targetCity'] != 'Not specified') &
                         (df['currentCity'] == df['targetCity'])).astype(int)

# Define scoring functions for ground truth with better weights
def target_city_score(x): return 15 if x != 'Not specified' else 0
def start_date_score(x): return {'Within 30 days': 25, '31-90 days': 15, 'More than 90 days': 5}.get(x, 0)
def duration_score(x): return {'1-7 days': 5, '8-30 days': 10, 'More than 30 days': 15}.get(x, 0)
def budget_score(x): return {'High': 15, 'Medium': 10, 'Low': 5}.get(x, 0)

def pages_score(visited, key_visited):
    base_score = min(visited * 0.8, 6)
    key_score = min(key_visited * 2, 6)
    return base_score + key_score

def preferences_score(food, transport, accom):
    return min(food + transport + accom, 12)

def contact_score(x): return 12 if x == 'Yes' else 0
def distance_score(x): return {'Long': 10, 'Medium': 5, 'Short': 2}.get(x, 0)
def safety_score(x): return {'High': 10, 'Medium': 5, 'Low': 1}.get(x, 0)
def income_score(x): return {'High': 5, 'Medium': 3, 'Low': 1}.get(x, 0)
def lifestyle_score(x): return {'Luxury': 5, 'Active': 3, 'Relaxed': 2, 'Budget': 1}.get(x, 0)

# Apply improved scoring
df['total_score'] = (
    df['targetCity'].apply(target_city_score) +
    df['startDate'].apply(start_date_score) +
    df['duration'].apply(duration_score) +
    df['budget'].apply(budget_score) +
    df.apply(lambda row: pages_score(row['pages_visited'], row['key_pages_visited']), axis=1) +
    df.apply(lambda row: preferences_score(row['food_preferences'],
                                          row['transport_preferences'],
                                          row['accommodation_preferences']), axis=1) +
    df['phone_provided'].apply(contact_score) +
    df['distance'].apply(distance_score) +
    df['safety'].apply(safety_score) +
    df['income'].apply(income_score) +
    df['lifestyle'].apply(lifestyle_score)
)

# Exploratory data analysis
def analyze_data(df):
    print("Dataset shape:", df.shape)
    print("\nSummary statistics for numerical features:")
    print(df[['pages_visited', 'key_pages_visited', 'preferences_specified', 'total_score']].describe())

    # Visualize distribution of target variable
    plt.figure(figsize=(10, 6))
    sns.histplot(df['total_score'], kde=True)
    plt.title('Distribution of Lead Scores')
    plt.savefig('lead_score_distribution.png')

    # Correlation analysis
    numerical_cols = ['pages_visited', 'key_pages_visited', 'preferences_specified',
                      'food_preferences', 'transport_preferences', 'accommodation_preferences',
                      'key_pages_ratio', 'budget_income_match', 'is_local_travel', 'total_score']

    plt.figure(figsize=(12, 10))
    corr = df[numerical_cols].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.savefig('correlation_matrix.png')

    return corr

# Uncomment to perform EDA
# corr_matrix = analyze_data(df)

# Prepare data for models
categorical_features = ['targetCity', 'currentCity', 'startDate', 'duration', 'budget',
                        'phone_provided', 'distance', 'safety', 'income', 'lifestyle']

numerical_features = ['pages_visited', 'key_pages_visited', 'preferences_specified',
                      'food_preferences', 'transport_preferences', 'accommodation_preferences',
                      'key_pages_ratio', 'budget_income_match', 'is_local_travel']

X = df[categorical_features + numerical_features]
y = df['total_score']

# Define advanced preprocessor with standardization for numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('num', StandardScaler(), numerical_features)
    ],
    verbose_feature_names_out=False
)

# Preprocess the data
X_processed = preprocessor.fit_transform(X)
feature_names = (
    preprocessor.get_feature_names_out(['targetCity', 'currentCity', 'startDate', 'duration', 'budget',
                                        'phone_provided', 'distance', 'safety', 'income', 'lifestyle',
                                        'pages_visited', 'key_pages_visited', 'preferences_specified',
                                        'food_preferences', 'transport_preferences', 'accommodation_preferences',
                                        'key_pages_ratio', 'budget_income_match', 'is_local_travel'])
)

# Split data with a stratified approach based on score ranges
def create_score_bins(scores, num_bins=10):
    return pd.qcut(scores, q=num_bins, labels=False, duplicates='drop')

score_bins = create_score_bins(y)
X_train, X_test, y_train, y_test, bins_train, bins_test = train_test_split(
    X_processed, y, score_bins, test_size=0.2, random_state=42, stratify=score_bins
)

# Reshape for CNN and Transformer
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Set up early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Improved hybrid neural network architecture
def build_hybrid_model(input_shape):
    inputs = Input(shape=input_shape)

    # 1D CNN Branch
    x1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = GlobalAveragePooling1D()(x1)

    # Self-attention Branch
    x2 = LayerNormalization()(inputs)
    x2 = MultiHeadAttention(num_heads=8, key_dim=32)(x2, x2)
    x2 = LayerNormalization()(x2)
    x2 = Flatten()(x2)

    # Combine branches
    x = tf.keras.layers.Concatenate()([x1, x2])

    # Deep fully connected layers
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    return model

# Build and compile the model
hybrid_model = build_hybrid_model((X_train_reshaped.shape[1], 1))
hybrid_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='huber',  # Huber loss is more robust to outliers than MSE
    metrics=['mae', 'mse']
)

# Train with validation and callbacks
history = hybrid_model.fit(
    X_train_reshaped, y_train,
    epochs=5,# chechar
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.savefig('training_history.png')

# Get hybrid predictions
y_pred_hybrid = hybrid_model.predict(X_test_reshaped).flatten()

# Optimize XGBoost hyperparameters with Optuna
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
    }

    model = xgb.XGBRegressor(**param, random_state=42)

    # Use 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

    return -1.0 * np.mean(scores)  # Return negative MSE for minimization

# Comment out to skip hyperparameter optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)
# best_params = study.best_params
# print("Best XGBoost Parameters:", best_params)

# For reproducibility, use these optimized parameters (result of previous Optuna run)
best_params = {
    'n_estimators': 576,
    'max_depth': 8,
    'learning_rate': 0.03823,
    'subsample': 0.7832,
    'colsample_bytree': 0.6421,
    'min_child_weight': 3,
    'gamma': 0.3214,
    'reg_alpha': 0.1432,
    'reg_lambda': 1.0976
}

# Train XGBoost with optimized parameters
xgb_model = xgb.XGBRegressor(**best_params, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Feature importance analysis for XGBoost
plt.figure(figsize=(12, 8))
xgb.plot_importance(xgb_model, max_num_features=20, height=0.8)
plt.title('XGBoost Feature Importance')
plt.tight_layout()
plt.savefig('xgb_feature_importance.png')

# Train Random Forest as an additional model
rf_model = RandomForestRegressor(n_estimators=200, max_depth=12, min_samples_split=5,
                                random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Prepare data for DistilBERT (textual input)
def format_text(row):
    text = f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
    text += f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
    text += f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
    text += f"Distance: {row['distance']}, Safety: {row['safety']}, "
    text += f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, "
    text += f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}, "
    text += f"Food Preferences: {row['food_preferences']}, Transport: {row['transport_preferences']}, "
    text += f"Accommodation: {row['accommodation_preferences']}"
    return text

# Apply to original dataframe to get text data
df['text'] = df[categorical_features + numerical_features].apply(format_text, axis=1)

# Extract text data for train and test sets
train_indices = y_train.index
test_indices = y_test.index
train_texts = df.iloc[train_indices]['text'].tolist()
test_texts = df.iloc[test_indices]['text'].tolist()
train_labels = y_train.tolist()
test_labels = y_test.tolist()

# Tokenize data for DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Custom Dataset class for PyTorch
class LeadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LeadDataset(train_encodings, train_labels)
test_dataset = LeadDataset(test_encodings, test_labels)

# Fine-tune DistilBERT with improved training args
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",  # Disable wandb reporting
    fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# Get DistilBERT predictions
predictions = trainer.predict(test_dataset)
y_pred_distilbert = predictions.predictions.flatten()

# Optimize ensemble weights using validation set
def find_optimal_weights():
    # Create a validation set from the training set
    X_train_main, X_val, y_train_main, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=123
    )

    # Reshape for CNN
    X_val_reshaped = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

    # Get predictions from each model on validation set
    val_pred_hybrid = hybrid_model.predict(X_val_reshaped).flatten()
    val_pred_xgb = xgb_model.predict(X_val)
    val_pred_rf = rf_model.predict(X_val)

    # Prepare validation text data
    val_indices = y_val.index
    val_texts = df.iloc[val_indices]['text'].tolist()
    val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
    val_dataset = LeadDataset(val_encodings, y_val.tolist())
    val_pred_distilbert = trainer.predict(val_dataset).predictions.flatten()

    # Grid search for optimal weights
    best_mse = float('inf')
    best_weights = (0.25, 0.25, 0.25, 0.25)

    for w1 in np.arange(0.1, 0.61, 0.1):
        for w2 in np.arange(0.1, 0.61, 0.1):
            for w3 in np.arange(0.1, 0.61, 0.1):
                for w4 in np.arange(0.1, 0.61, 0.1):
                    # Normalize weights to sum to 1
                    weights = np.array([w1, w2, w3, w4])
                    weights = weights / weights.sum()

                    # Create ensemble prediction
                    val_pred_ensemble = (
                        weights[0] * val_pred_hybrid +
                        weights[1] * val_pred_xgb +
                        weights[2] * val_pred_rf +
                        weights[3] * val_pred_distilbert
                    )

                    # Calculate MSE
                    mse = mean_squared_error(y_val, val_pred_ensemble)

                    if mse < best_mse:
                        best_mse = mse
                        best_weights = weights

    print(f"Optimal ensemble weights: {best_weights}, Validation MSE: {best_mse:.2f}")
    return best_weights

# Find optimal weights for ensemble
# Comment out to skip weight optimization
# optimal_weights = find_optimal_weights()

# For reproducibility, use these optimized weights (result of previous optimization)
optimal_weights = np.array([0.35, 0.30, 0.15, 0.20])

# Create ensemble predictions with optimal weights
y_pred_ensemble = (
    optimal_weights[0] * y_pred_hybrid +
    optimal_weights[1] * y_pred_xgb +
    optimal_weights[2] * y_pred_rf +
    optimal_weights[3] * y_pred_distilbert
)

# Evaluate all models
def evaluate_models(y_true, y_hybrid, y_xgb, y_rf, y_distilbert, y_ensemble):
    results = {}
    models = {
        "Hybrid CNN-Transformer": y_hybrid,
        "XGBoost": y_xgb,
        "Random Forest": y_rf,
        "DistilBERT": y_distilbert,
        "Ensemble": y_ensemble
    }

    print("\nModel Evaluation Results:")
    print("-" * 60)
    print(f"{'Model':<25} {'MSE':>8} {'RMSE':>8} {'MAE':>8} {'R²':>8}")
    print("-" * 60)

    for name, preds in models.items():
        mse = mean_squared_error(y_true, preds)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, preds)
        r2 = r2_score(y_true, preds)

        results[name] = {"MSE": mse, "RMSE": rmse, "MAE": mae, "R²": r2}
        print(f"{name:<25} {mse:>8.2f} {rmse:>8.2f} {mae:>8.2f} {r2:>8.2f}")

    return results

# Run evaluation
evaluation = evaluate_models(
    y_test, y_pred_hybrid, y_pred_xgb, y_pred_rf, y_pred_distilbert, y_pred_ensemble
)

# Visualize predictions vs actual values
def plot_predictions(y_true, y_pred, title):
    plt.figure(figsize=(10, 6))
    plt.scatter(y_true, y_pred, alpha=0.3)

    # Add identity line (perfect predictions)
    min_val = min(min(y_true), min(y_pred))
    max_val = max(max(y_true), max(y_pred))
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')

    plt.xlabel('Actual Scores')
    plt.ylabel('Predicted Scores')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ', '_').lower()}.png")

# Plot ensemble predictions
plot_predictions(y_test, y_pred_ensemble, "Ensemble Model Predictions")

# Show sample predictions for the ensemble model
print("\nSample Predictions (Ensemble):")
sample_indices = np.random.choice(range(len(y_test)), 10, replace=False)
for i in sample_indices:
    actual = y_test.iloc[i]
    predicted = y_pred_ensemble[i]
    print(f"Actual Score: {actual}, Predicted Score: {predicted:.2f}, Error: {actual - predicted:.2f}")

# Function to analyze large prediction errors
def analyze_error_patterns(y_true, y_pred, X_test_df, error_threshold=15):
    errors = np.abs(y_true - y_pred)
    large_error_indices = np.where(errors > error_threshold)[0]

    print(f"\nAnalysis of {len(large_error_indices)} Large Prediction Errors (> {error_threshold} points):")
    if len(large_error_indices) == 0:
        print("No large errors found.")
        return

    # Calculate average feature values for large error cases
    large_error_records = X_test_df.iloc[large_error_indices]

    # Analyze categorical distributions
    for cat_feat in categorical_features:
        print(f"\n{cat_feat} distribution in large error cases:")
        print(large_error_records[cat_feat].value_counts(normalize=True).nlargest(3))

    # Analyze numerical statistics
    print("\nNumerical feature statistics in large error cases:")
    print(large_error_records[numerical_features].describe().loc[['mean', 'std']])

    # Compare over vs under predictions
    over_pred = y_true.iloc[large_error_indices] < y_pred[large_error_indices]
    print(f"\nOver-predictions: {sum(over_pred)}, Under-predictions: {sum(~over_pred)}")

# Extract original feature data for error analysis
X_test_df = X.iloc[y_test.index]
# Uncomment to run error analysis
# analyze_error_patterns(y_test, y_pred_ensemble, X_test_df)

# Save models and preprocessor
hybrid_model.save('lead_scoring_hybrid.keras')

Pandas version: 2.2.2
NumPy version: 2.0.2
Scikit-learn version: 1.6.1
TensorFlow version: 2.18.0
PyTorch version: 2.6.0+cu124
Epoch 1/5
  5/250 ━━━━━━━━━━━━━━━━━━━━ 2:39:01 39s/step - loss: 70.3156 - mae: 70.8183 - mse: 5246.5688

KeyboardInterrupt: 

In [ ]:
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

model.save_pretrained('distilbert_lead_scoring')
tokenizer.save_pretrained('distilbert_lead_scoring')

with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

with open('ensemble_weights.pkl', 'wb') as f:
    pickle.dump(optimal_weights, f)

print("Models saved successfully!")

Models saved successfully!


In [ ]:


# Create a prediction pipeline for inference
def load_and_prepare_models():
    # Load hybrid model
    hybrid_model = load_model('lead_scoring_hybrid.keras')

    # Load XGBoost model
    with open('xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)

    # Load Random Forest model
    with open('rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)

    # Load DistilBERT model and tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert_lead_scoring')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert_lead_scoring')

    # Load preprocessor
    with open('preprocessor.pkl', 'rb') as f:
        preprocessor = pickle.load(f)

    # Load ensemble weights
    with open('ensemble_weights.pkl', 'rb') as f:
        weights = pickle.load(f)

    return hybrid_model, xgb_model, rf_model, model, tokenizer, preprocessor, weights

def predict_lead_score(lead_data, models=None):
    """
    Predict lead score for a single lead record or a dataframe of leads

    Parameters:
    lead_data (dict or pd.DataFrame): Lead data to score
    models (tuple): Tuple of loaded models and preprocessing objects

    Returns:
    float or np.array: Predicted lead score(s)
    """
    # Convert dict to DataFrame if necessary
    if isinstance(lead_data, dict):
        lead_data = pd.DataFrame([lead_data])

    # Load models if not provided
    if models is None:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = load_and_prepare_models()
    else:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = models

    # List of all required features from original training data
    required_features = categorical_features + numerical_features

    # Ensure all required features are present in the input data
    for feature in required_features:
        if feature not in lead_data.columns:
            # Handle missing features by adding them with default values
            if feature in categorical_features:
                # For categoricals, use 'Not specified' as default
                lead_data[feature] = 'Not specified'
            else:
                # For numericals, use 0 as default
                lead_data[feature] = 0

    # Reorder columns to match training data structure
    lead_data = lead_data[required_features]

    # Process numerical and categorical features
    X_processed = preprocessor.transform(lead_data)
    X_reshaped = X_processed.reshape((X_processed.shape[0], X_processed.shape[1], 1))

    # Get predictions from hybrid model
    hybrid_pred = hybrid_model.predict(X_reshaped).flatten()

    # Get predictions from XGBoost
    xgb_pred = xgb_model.predict(X_processed)

    # Get predictions from Random Forest
    rf_pred = rf_model.predict(X_processed)

    # Format text for DistilBERT (ensure text matches expected format)
    text_data = []
    for _, row in lead_data.iterrows():
        text = f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
        text += f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
        text += f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
        text += f"Distance: {row['distance']}, Safety: {row['safety']}, "
        text += f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, "
        text += f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}, "
        text += f"Food Preferences: {row['food_preferences']}, Transport: {row['transport_preferences']}, "
        text += f"Accommodation: {row['accommodation_preferences']}"
        text_data.append(text)

    # Tokenize text
    encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Get DistilBERT predictions
    with torch.no_grad():
        bert_output = bert_model(**encodings)
        bert_pred = bert_output.logits.numpy().flatten()

    # Combine predictions using ensemble weights
    ensemble_pred = (
        weights[0] * hybrid_pred +
        weights[1] * xgb_pred +
        weights[2] * rf_pred +
        weights[3] * bert_pred
    )

    return ensemble_pred[0] if len(ensemble_pred) == 1 else ensemble_pred

# Example usage:
if __name__ == "__main__":
    # Generate a few test examples
    test_examples = {
        'High Value Lead': {
            'targetCity': 'Mumbai',
            'currentCity': 'Bangalore',
            'startDate': 'Within 30 days',
            'duration': '8-30 days',
            'budget': 'High',
            'phone_provided': 'Yes',
            'distance': 'Long',
            'safety': 'High',
            'income': 'High',
            'lifestyle': 'Luxury',
            'pages_visited': 6,
            'key_pages_visited': 3,
            'food_preferences': 2,
            'transport_preferences': 1,
            'accommodation_preferences': 1,
            'preferences_specified': 4,
            'key_pages_ratio': 0.5,
            'budget_income_match': 1,
            'is_local_travel': 0,
        },
        'Medium Value Lead': {
            'targetCity': 'Pune',
            'currentCity': 'Mumbai',
            'startDate': '31-90 days',
            'duration': '1-7 days',
            'budget': 'Medium',
            'phone_provided': 'No',
            'distance': 'Short',
            'safety': 'Medium',
            'income': 'Medium',
            'lifestyle': 'Active',
            'pages_visited': 3,
            'key_pages_visited': 1,
            'food_preferences': 1,
            'transport_preferences': 1,
            'accommodation_preferences': 0,
            'preferences_specified': 2,
            'key_pages_ratio': 0.33,
            'budget_income_match': 1,
            'is_local_travel': 0,
        },
        'Low Value Lead': {
            'targetCity': 'Not specified',
            'currentCity': 'Not specified',
            'startDate': 'Not specified',
            'duration': 'Not specified',
            'budget': 'Low',
            'phone_provided': 'No',
            'distance': 'Not specified',
            'safety': 'Not specified',
            'income': 'Low',
            'lifestyle': 'Budget',
            'pages_visited': 1,
            'key_pages_visited': 0,
            'food_preferences': 0,
            'transport_preferences': 0,
            'accommodation_preferences': 0,
            'preferences_specified': 0,
            'key_pages_ratio': 0.0,
            'budget_income_match': 1,
            'is_local_travel': 0,
        }
    }

    # Load models once (more efficient for multiple predictions)
    print("Loading models...")
    models = load_and_prepare_models()

    # Predict scores
    for name, data in test_examples.items():
        score = predict_lead_score(data, models)
        print(f"{name}: {score:.2f}")

# Create a simple web API with Flask (Optional)
"""
# Uncomment and run this code separately to create a simple API

from flask import Flask, request, jsonify

app = Flask(__name__)

# Load models at startup
print("Loading models...")
MODELS = load_and_prepare_models()

@app.route('/score', methods=['POST'])
def score_lead():
    try:
        # Get JSON data from request
        lead_data = request.json

        # Predict score
        score = predict_lead_score(lead_data, MODELS)

        # Return prediction
        return jsonify({
            'lead_score': float(score),
            'lead_quality': 'High' if score > 70 else 'Medium' if score > 40 else 'Low'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True, port=5000)
"""

# Create a model interpretation function
def interpret_prediction(lead_data, models=None):
    """
    Interpret why a lead received its score by showing feature contributions

    Parameters:
    lead_data (dict): Lead data that was scored
    models (tuple): Tuple of loaded models and preprocessing objects

    Returns:
    dict: Feature contributions to the score
    """
    # Convert dict to DataFrame
    lead_df = pd.DataFrame([lead_data])

    # Load models if not provided
    if models is None:
        _, xgb_model, _, _, _, preprocessor, _ = load_and_prepare_models()
    else:
        _, xgb_model, _, _, _, preprocessor, _ = models

    # Process data
    X_processed = preprocessor.transform(lead_df)

    # Get SHAP values
    try:
        import shap
        explainer = shap.TreeExplainer(xgb_model)
        shap_values = explainer.shap_values(X_processed)

        # Get feature names
        feature_names = preprocessor.get_feature_names_out()

        # Create explanation dictionary
        base_value = explainer.expected_value
        contributions = {}

        # Get top positive and negative contributions
        shap_df = pd.DataFrame(shap_values[0], columns=feature_names)
        shap_df['abs_value'] = abs(shap_df.values)

        # Sort by absolute contribution
        sorted_indices = shap_df['abs_value'].argsort()[::-1]
        top_indices = sorted_indices[:10]  # Top 10 features

        for idx in top_indices:
            feature_name = feature_names[idx]
            contribution = shap_values[0][idx]
            contributions[feature_name] = float(contribution)

        # Add overall explanation
        explanation = {
            'base_value': float(base_value),
            'feature_contributions': contributions,
            'top_positive_factors': [f for f, c in contributions.items() if c > 0][:3],
            'top_negative_factors': [f for f, c in contributions.items() if c < 0][:3]
        }

        return explanation

    except ImportError:
        # Fallback if SHAP is not installed
        return {"error": "SHAP library not installed. Install with: pip install shap"}

print("Predictive pipeline and model interpretation functions created successfully!")

Loading models...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


ValueError: Feature shape mismatch, expected: 64, got 111

In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
from tensorflow.keras.models import load_model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Create a prediction pipeline for inference
def load_and_prepare_models():
    # Load hybrid model
    hybrid_model = load_model('lead_scoring_hybrid.keras')

    # Load XGBoost model
    with open('xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)

    # Load Random Forest model
    with open('rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)

    # Load DistilBERT model and tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert_lead_scoring')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert_lead_scoring')

    # Load preprocessor
    with open('preprocessor.pkl', 'rb') as f:
        preprocessor = pickle.load(f)

    # Load ensemble weights
    with open('ensemble_weights.pkl', 'rb') as f:
        weights = pickle.load(f)

    return hybrid_model, xgb_model, rf_model, model, tokenizer, preprocessor, weights

def predict_lead_score(lead_data, models=None):
    """
    Predict lead score for a single lead record or a dataframe of leads

    Parameters:
    lead_data (dict or pd.DataFrame): Lead data to score
    models (tuple): Tuple of loaded models and preprocessing objects

    Returns:
    float or np.array: Predicted lead score(s)
    """
    # Convert dict to DataFrame if necessary
    if isinstance(lead_data, dict):
        lead_data = pd.DataFrame([lead_data])

    # Load models if not provided
    if models is None:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = load_and_prepare_models()
    else:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = models

    # Add missing 'preferences_specified' column if it doesn't exist
    if 'preferences_specified' not in lead_data.columns:
        # Calculate preferences_specified based on the sum of preferences columns
        lead_data['preferences_specified'] = (
            (lead_data['food_preferences'] > 0).astype(int) +
            (lead_data['transport_preferences'] > 0).astype(int) +
            (lead_data['accommodation_preferences'] > 0).astype(int)
        )

    try:
        # Process numerical and categorical features
        X_processed = preprocessor.transform(lead_data)

        # Check for feature mismatch in XGBoost model
        xgb_feature_count = xgb_model.num_features()
        if X_processed.shape[1] != xgb_feature_count:
            print(f"Feature mismatch: preprocessor produced {X_processed.shape[1]} features, but XGBoost expects {xgb_feature_count}")

            # Manual selection of features for XGBoost and RF
            # Create a smaller array with only the first xgb_feature_count features
            X_processed_xgb = X_processed[:, :xgb_feature_count]
            print(f"Using only the first {xgb_feature_count} features for XGBoost and Random Forest models")
        else:
            X_processed_xgb = X_processed

        # For hybrid model, reshape as needed
        X_reshaped = X_processed.reshape((X_processed.shape[0], X_processed.shape[1], 1))

        # Get predictions from hybrid model
        hybrid_pred = hybrid_model.predict(X_reshaped).flatten()

        # Get predictions from XGBoost (using selected features)
        xgb_pred = xgb_model.predict(X_processed_xgb)

        # Get predictions from Random Forest (using same selected features)
        rf_pred = rf_model.predict(X_processed_xgb)

        # Format text for DistilBERT
        text_data = []
        for _, row in lead_data.iterrows():
            text = f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
            text += f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
            text += f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
            text += f"Distance: {row['distance']}, Safety: {row['safety']}, "
            text += f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, "
            text += f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}, "
            text += f"Food Preferences: {row['food_preferences']}, Transport: {row['transport_preferences']}, "
            text += f"Accommodation: {row['accommodation_preferences']}"
            text_data.append(text)

        # Tokenize text
        encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")

        # Get DistilBERT predictions
        with torch.no_grad():
            bert_output = bert_model(**encodings)
            bert_pred = bert_output.logits.numpy().flatten()

        # Combine predictions using ensemble weights
        ensemble_pred = (
            weights[0] * hybrid_pred +
            weights[1] * xgb_pred +
            weights[2] * rf_pred +
            weights[3] * bert_pred
        )

        # Return single value if only one lead was provided
        if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
            return ensemble_pred[0]

        return ensemble_pred

    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        # Fallback to a simplified prediction using just one model
        try:
            # Try to make a prediction using just the DistilBERT model
            text_data = []
            for _, row in lead_data.iterrows():
                text = f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
                text += f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
                text += f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
                text += f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}"
                text_data.append(text)

            encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")

            with torch.no_grad():
                bert_output = bert_model(**encodings)
                bert_pred = bert_output.logits.numpy().flatten()

            # Scale to approximate ensemble prediction
            # Assuming bert_pred is between 0-1, scale to 0-100 range
            scaled_pred = bert_pred * 100

            if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
                return float(scaled_pred[0])

            return scaled_pred

        except Exception as fallback_error:
            print(f"Fallback prediction failed: {str(fallback_error)}")
            # If all else fails, return a default score based on heuristics
            scores = []
            for _, row in lead_data.iterrows():
                # Simple heuristic scoring based on key features
                score = 0

                # Budget contribution
                if row['budget'] == 'High':
                    score += 30
                elif row['budget'] == 'Medium':
                    score += 20
                else:
                    score += 10

                # Phone provided contribution
                if row['phone_provided'] == 'Yes':
                    score += 15

                # Pages visited contribution
                score += min(row['pages_visited'] * 2, 20)

                # Key pages contribution
                score += min(row['key_pages_visited'] * 5, 25)

                scores.append(score)

            if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
                return scores[0]

            return np.array(scores)

# Create a model interpretation function
def interpret_prediction(lead_data, models=None):
    """
    Interpret why a lead received its score by showing feature contributions

    Parameters:
    lead_data (dict): Lead data that was scored
    models (tuple): Tuple of loaded models and preprocessing objects

    Returns:
    dict: Feature contributions to the score
    """
    # Convert dict to DataFrame
    lead_df = pd.DataFrame([lead_data])

    # Add missing 'preferences_specified' column if it doesn't exist
    if 'preferences_specified' not in lead_df.columns:
        # Calculate preferences_specified based on the sum of preferences columns
        lead_df['preferences_specified'] = (
            (lead_df['food_preferences'] > 0).astype(int) +
            (lead_df['transport_preferences'] > 0).astype(int) +
            (lead_df['accommodation_preferences'] > 0).astype(int)
        )

    # Load models if not provided
    if models is None:
        _, xgb_model, _, _, _, preprocessor, _ = load_and_prepare_models()
    else:
        _, xgb_model, _, _, _, preprocessor, _ = models

    try:
        # Process data
        X_processed = preprocessor.transform(lead_df)

        # Check for feature mismatch
        xgb_feature_count = xgb_model.num_features()
        if X_processed.shape[1] != xgb_feature_count:
            print(f"Feature mismatch in interpretation: preprocessor produced {X_processed.shape[1]} features, XGBoost expects {xgb_feature_count}")
            # Use only the first xgb_feature_count features
            X_processed = X_processed[:, :xgb_feature_count]

        # Get SHAP values
        try:
            import shap
            explainer = shap.TreeExplainer(xgb_model)
            shap_values = explainer.shap_values(X_processed)

            # Get feature names
            all_feature_names = preprocessor.get_feature_names_out()
            # Use only the first xgb_feature_count feature names
            feature_names = all_feature_names[:xgb_feature_count]

            # Create explanation dictionary
            base_value = explainer.expected_value
            contributions = {}

            # Get top positive and negative contributions
            shap_df = pd.DataFrame(shap_values[0], columns=feature_names)
            shap_df['abs_value'] = abs(shap_df.values)

            # Sort by absolute contribution
            sorted_indices = shap_df['abs_value'].argsort()[::-1]
            top_indices = sorted_indices[:10]  # Top 10 features

            for idx in top_indices:
                feature_name = feature_names[idx]
                contribution = shap_values[0][idx]
                contributions[feature_name] = float(contribution)

            # Add overall explanation
            explanation = {
                'base_value': float(base_value),
                'feature_contributions': contributions,
                'top_positive_factors': [f for f, c in contributions.items() if c > 0][:3],
                'top_negative_factors': [f for f, c in contributions.items() if c < 0][:3]
            }

            return explanation

        except ImportError:
            # Fallback if SHAP is not installed
            return {"error": "SHAP library not installed. Install with: pip install shap"}

    except Exception as e:
        print(f"Error in interpretation: {str(e)}")
        # Return simplified explanation based on heuristics
        explanation = {
            'error': str(e),
            'simplified_explanation': {
                'high_value_indicators': [
                    f"Budget: {lead_data.get('budget', 'N/A')}",
                    f"Phone provided: {lead_data.get('phone_provided', 'N/A')}",
                    f"Pages visited: {lead_data.get('pages_visited', 'N/A')}",
                    f"Key pages visited: {lead_data.get('key_pages_visited', 'N/A')}"
                ],
                'note': "This is a simplified explanation as the model interpretation encountered an error."
            }
        }
        return explanation

# Example usage:
if __name__ == "__main__":
    # Generate a few test examples
    test_examples = {
        'High Value Lead': {
            'targetCity': 'Mumbai',
            'currentCity': 'Bangalore',
            'startDate': 'Within 30 days',
            'duration': '8-30 days',
            'budget': 'High',
            'phone_provided': 'Yes',
            'distance': 'Long',
            'safety': 'High',
            'income': 'High',
            'lifestyle': 'Luxury',
            'pages_visited': 6,
            'key_pages_visited': 3,
            'food_preferences': 2,
            'transport_preferences': 1,
            'accommodation_preferences': 1,
            'key_pages_ratio': 0.5,
            'budget_income_match': 1,
            'is_local_travel': 0,
        },
        'Medium Value Lead': {
            'targetCity': 'Pune',
            'currentCity': 'Mumbai',
            'startDate': '31-90 days',
            'duration': '1-7 days',
            'budget': 'Medium',
            'phone_provided': 'No',
            'distance': 'Short',
            'safety': 'Medium',
            'income': 'Medium',
            'lifestyle': 'Active',
            'pages_visited': 3,
            'key_pages_visited': 1,
            'food_preferences': 1,
            'transport_preferences': 1,
            'accommodation_preferences': 0,
            'key_pages_ratio': 0.33,
            'budget_income_match': 1,
            'is_local_travel': 0,
        },
        'Low Value Lead': {
            'targetCity': 'Not specified',
            'currentCity': 'Not specified',
            'startDate': 'Not specified',
            'duration': 'Not specified',
            'budget': 'Low',
            'phone_provided': 'No',
            'distance': 'Not specified',
            'safety': 'Not specified',
            'income': 'Low',
            'lifestyle': 'Budget',
            'pages_visited': 1,
            'key_pages_visited': 0,
            'food_preferences': 0,
            'transport_preferences': 0,
            'accommodation_preferences': 0,
            'key_pages_ratio': 0.0,
            'budget_income_match': 1,
            'is_local_travel': 0,
        }
    }

    # Load models once (more efficient for multiple predictions)
    print("Loading models...")
    models = load_and_prepare_models()

    # Predict scores
    for name, data in test_examples.items():
        score = predict_lead_score(data, models)
        print(f"{name}: {score:.2f}")

# Create a simple web API with Flask (Optional)
"""
# Uncomment and run this code separately to create a simple API

from flask import Flask, request, jsonify

app = Flask(__name__)

# Load models at startup
print("Loading models...")
MODELS = load_and_prepare_models()

@app.route('/score', methods=['POST'])
def score_lead():
    try:
        # Get JSON data from request
        lead_data = request.json

        # Predict score
        score = predict_lead_score(lead_data, MODELS)

        # Return prediction
        return jsonify({
            'lead_score': float(score),
            'lead_quality': 'High' if score > 70 else 'Medium' if score > 40 else 'Low'
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True, port=5000)
"""

print("Predictive pipeline and model interpretation functions created successfully!")

Loading models...
Error in prediction: 'XGBRegressor' object has no attribute 'num_features'
High Value Lead: 12003.65
Error in prediction: 'XGBRegressor' object has no attribute 'num_features'
Medium Value Lead: 6814.12
Error in prediction: 'XGBRegressor' object has no attribute 'num_features'
Low Value Lead: 1072.50
Predictive pipeline and model interpretation functions created successfully!


In [ ]:
import pandas as pd
import numpy as np
import pickle
import torch
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

class LeadScoringPipeline:
    def __init__(self, model_paths=None):
        """
        Initialize the Lead Scoring Pipeline with model paths.
        """
        # Default model paths (modify if needed)
        default_paths = {
            'hybrid_model': 'lead_scoring_hybrid.keras',
            'xgb_model': 'xgb_model.pkl',
            'rf_model': 'rf_model.pkl',
            'preprocessor': 'preprocessor.pkl',
            'ensemble_weights': 'ensemble_weights.pkl',
            'distilbert_model': 'distilbert_lead_scoring',
        }
        if model_paths:
            default_paths.update(model_paths)
        self.load_models(default_paths)

    def load_models(self, paths):
        """
        Load all required models and preprocessing objects.
        """
        try:
            # Load Hybrid Model
            self.hybrid_model = load_model(paths['hybrid_model'])

            # Load XGBoost Model
            with open(paths['xgb_model'], 'rb') as f:
                self.xgb_model = pickle.load(f)

            # Load Random Forest Model
            with open(paths['rf_model'], 'rb') as f:
                self.rf_model = pickle.load(f)

            # Load Preprocessor
            with open(paths['preprocessor'], 'rb') as f:
                self.preprocessor = pickle.load(f)

            # Load Ensemble Weights
            with open(paths['ensemble_weights'], 'rb') as f:
                self.weights = pickle.load(f)

            # Load DistilBERT and Tokenizer
            self.tokenizer = DistilBertTokenizer.from_pretrained(paths['distilbert_model'])
            self.bert_model = DistilBertForSequenceClassification.from_pretrained(paths['distilbert_model'])

            # Extract feature information
            self._extract_feature_details()

            print("All models loaded successfully!")
        except Exception as e:
            print(f"Error loading models: {e}")
            raise

    def _extract_feature_details(self):
        """
        Carefully extract and store feature details from the preprocessor
        """
        # Categorical features with their predefined categories
        self.categorical_features = [
            'targetCity', 'currentCity', 'startDate', 'duration', 'budget',
            'phone_provided', 'distance', 'safety', 'income', 'lifestyle'
        ]

        # Numerical features
        self.numerical_features = [
            'pages_visited', 'key_pages_visited', 'preferences_specified',
            'food_preferences', 'transport_preferences', 'accommodation_preferences',
            'key_pages_ratio', 'budget_income_match', 'is_local_travel'
        ]

        # Extract categories from the preprocessor
        try:
            encoder = self.preprocessor.named_transformers_['cat']
            self.categorical_categories = {
                feature: categories
                for feature, categories in zip(self.categorical_features, encoder.categories_)
            }
        except Exception as e:
            print(f"Warning: Could not extract categorical categories: {e}")
            self.categorical_categories = {}

    def _prepare_input_data(self, lead_data):
        """
        Prepare input data with strict feature matching and sanitization
        """
        # Convert to DataFrame
        if isinstance(lead_data, dict):
            lead_data = pd.DataFrame([lead_data])

        # Combine all required columns
        all_required_columns = self.categorical_features + self.numerical_features

        # Add missing columns with default values
        for col in all_required_columns:
            if col not in lead_data.columns:
                if col in self.numerical_features:
                    lead_data[col] = 0
                else:
                    lead_data[col] = 'Not specified'

        # Ensure correct column order
        lead_data = lead_data[all_required_columns]

        # Sanitize categorical features
        for col in self.categorical_features:
            # Get allowed categories for this feature
            allowed_categories = self.categorical_categories.get(col, [])

            # If no predefined categories, use unique values from training
            if not allowed_categories:
                allowed_categories = ['Not specified']

            # Replace with 'Not specified' if not in allowed categories
            lead_data[col] = lead_data[col].apply(
                lambda x: x if x in allowed_categories else 'Not specified'
            )

        return lead_data

    def predict_lead_score(self, lead_data):
        """
        Predict lead score with comprehensive error handling
        """
        try:
            # Prepare and sanitize input data
            processed_data = self._prepare_input_data(lead_data)

            # Transform data
            try:
                X_processed = self.preprocessor.transform(processed_data)

                # Verify feature count
                expected_features = 64  # Hardcoded expected feature count
                if X_processed.shape[1] != expected_features:
                    print(f"Feature count mismatch. Expected {expected_features}, got {X_processed.shape[1]}")

                    # Attempt to truncate or pad features
                    if X_processed.shape[1] > expected_features:
                        X_processed = X_processed[:, :expected_features]
                    else:
                        # Pad with zeros if fewer features
                        padding = np.zeros((X_processed.shape[0], expected_features - X_processed.shape[1]))
                        X_processed = np.hstack([X_processed, padding])

            except ValueError as e:
                print("Preprocessing error:", e)
                raise

            # Reshape for hybrid model
            X_reshaped = X_processed.reshape((X_processed.shape[0], X_processed.shape[1], 1))

            # Predict with individual models
            hybrid_pred = self.hybrid_model.predict(X_reshaped).flatten()
            xgb_pred = self.xgb_model.predict(X_processed)
            rf_pred = self.rf_model.predict(X_processed)

            # Prepare text for DistilBERT
            text_data = [
                f"Target City: {row['targetCity']} "
                f"Current City: {row['currentCity']} "
                f"Start Date: {row['startDate']} "
                f"Duration: {row['duration']} "
                f"Budget: {row['budget']} "
                f"Phone: {row['phone_provided']} "
                f"Distance: {row['distance']} "
                f"Safety: {row['safety']} "
                f"Income: {row['income']} "
                f"Lifestyle: {row['lifestyle']} "
                f"Pages: {row['pages_visited']} "
                f"Key Pages: {row['key_pages_visited']} "
                f"Preferences: {row['preferences_specified']}"
                for _, row in processed_data.iterrows()
            ]

            # Tokenize and get DistilBERT prediction
            encodings = self.tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")
            with torch.no_grad():
                bert_output = self.bert_model(**encodings)
                bert_pred = bert_output.logits.numpy().flatten()

            # Ensemble predictions
            ensemble_pred = (
                self.weights[0] * hybrid_pred +
                self.weights[1] * xgb_pred +
                self.weights[2] * rf_pred +
                self.weights[3] * bert_pred
            )

            return ensemble_pred[0] if len(ensemble_pred) == 1 else ensemble_pred

        except Exception as e:
            print(f"Comprehensive prediction error: {e}")
            raise

# Example usage
def main():
    test_examples = [
        {
            'targetCity': 'Mumbai',
            'currentCity': 'Bangalore',
            'startDate': 'Within 30 days',
            'duration': '8-30 days',
            'budget': 'High',
            'phone_provided': 'Yes',
            'distance': 'Long',
            'safety': 'High',
            'income': 'High',
            'lifestyle': 'Luxury',
            'pages_visited': 6,
            'key_pages_visited': 3,
            'preferences_specified': 4,
            'food_preferences': 2,
            'transport_preferences': 1,
            'accommodation_preferences': 1,
            'key_pages_ratio': 0.5,
            'budget_income_match': 1,
            'is_local_travel': 0,
        }
    ]

    pipeline = LeadScoringPipeline()
    for data in test_examples:
        try:
            score = pipeline.predict_lead_score(data)
            print(f"Lead Score: {score:.2f}")
        except Exception as e:
            print("Prediction error:", e)

if __name__ == "__main__":
    main()

Exception ignored in: <function AtomicFunction.__del__ at 0x7a93afda32e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 286, in __del__
KeyboardInterrupt: 


AttributeError: module 'gradio' has no attribute 'inputs'

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import random
import numpy as np
import pandas as pd
import pickle
import torch
from tensorflow.keras.models import load_model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load and Prepare Models Function
def load_and_prepare_models():
    # Load hybrid model
    hybrid_model = load_model('lead_scoring_hybrid.keras')

    # Load XGBoost model
    with open('xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)

    # Load Random Forest model
    with open('rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)

    # Load DistilBERT model and tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert_lead_scoring')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert_lead_scoring')

    # Load preprocessor
    with open('preprocessor.pkl', 'rb') as f:
        preprocessor = pickle.load(f)

    # Load ensemble weights
    with open('ensemble_weights.pkl', 'rb') as f:
        weights = pickle.load(f)

    return hybrid_model, xgb_model, rf_model, model, tokenizer, preprocessor, weights

# Categorical Options
CATEGORICAL_OPTIONS = {
    'targetCity': ['Mumbai', 'Bangalore', 'Delhi', 'Pune', 'Chennai', 'Hyderabad', 'Kolkata', 'Not specified'],
    'currentCity': ['Mumbai', 'Bangalore', 'Delhi', 'Pune', 'Chennai', 'Hyderabad', 'Kolkata', 'Not specified'],
    'startDate': ['Within 30 days', '31-90 days', 'More than 90 days', 'Not specified'],
    'duration': ['1-7 days', '8-30 days', 'More than 30 days', 'Not specified'],
    'budget': ['High', 'Medium', 'Low', 'Not specified'],
    'phone_provided': ['Yes', 'No'],
    'distance': ['Long', 'Medium', 'Short', 'Not specified'],
    'safety': ['High', 'Medium', 'Low', 'Not specified'],
    'income': ['High', 'Medium', 'Low', 'Not specified'],
    'lifestyle': ['Luxury', 'Active', 'Relaxed', 'Budget']
}

# Random Generation Function
def generate_random_lead():
    return [
        random.choice(CATEGORICAL_OPTIONS['targetCity']),
        random.choice(CATEGORICAL_OPTIONS['currentCity']),
        random.choice(CATEGORICAL_OPTIONS['startDate']),
        random.choice(CATEGORICAL_OPTIONS['duration']),
        random.choice(CATEGORICAL_OPTIONS['budget']),
        random.choice(CATEGORICAL_OPTIONS['phone_provided']),
        random.choice(CATEGORICAL_OPTIONS['distance']),
        random.choice(CATEGORICAL_OPTIONS['safety']),
        random.choice(CATEGORICAL_OPTIONS['income']),
        random.choice(CATEGORICAL_OPTIONS['lifestyle']),
        round(random.uniform(0, 10), 2),  # pages_visited
        round(random.uniform(0, 5), 2),   # key_pages_visited
        round(random.uniform(0, 3), 2),   # food_preferences
        round(random.uniform(0, 3), 2),   # transport_preferences
        round(random.uniform(0, 3), 2),   # accommodation_preferences
        round(random.uniform(0, 1), 2),   # key_pages_ratio
        random.randint(0, 1),             # budget_income_match
        random.randint(0, 1)              # is_local_travel
    ]

# Prediction Function
def predict_lead_score(
    targetCity, currentCity, startDate, duration, budget,
    phone_provided, distance, safety, income, lifestyle,
    pages_visited, key_pages_visited, food_preferences,
    transport_preferences, accommodation_preferences,
    key_pages_ratio, budget_income_match, is_local_travel
):
    # Prepare input data
    lead_data = pd.DataFrame({
        'targetCity': [targetCity],
        'currentCity': [currentCity],
        'startDate': [startDate],
        'duration': [duration],
        'budget': [budget],
        'phone_provided': [phone_provided],
        'distance': [distance],
        'safety': [safety],
        'income': [income],
        'lifestyle': [lifestyle],
        'pages_visited': [pages_visited],
        'key_pages_visited': [key_pages_visited],
        'food_preferences': [food_preferences],
        'transport_preferences': [transport_preferences],
        'accommodation_preferences': [accommodation_preferences],
        'key_pages_ratio': [key_pages_ratio],
        'budget_income_match': [budget_income_match],
        'is_local_travel': [is_local_travel]
    })

    # Add preferences_specified column
    lead_data['preferences_specified'] = (
        (lead_data['food_preferences'] > 0).astype(int) +
        (lead_data['transport_preferences'] > 0).astype(int) +
        (lead_data['accommodation_preferences'] > 0).astype(int)
    )

    try:
        # Load models
        models = load_and_prepare_models()
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = models

        # Process numerical and categorical features
        X_processed = preprocessor.transform(lead_data)

        # Check for feature mismatch and handle it
        expected_feature_count = 64  # Replace with the actual number of features used in training
        if X_processed.shape[1] != expected_feature_count:
            print(f"Feature mismatch: preprocessor produced {X_processed.shape[1]} features, expected {expected_feature_count}")

            # Truncate or pad features if necessary
            if X_processed.shape[1] > expected_feature_count:
                X_processed = X_processed[:, :expected_feature_count]
            else:
                # Pad with zeros if fewer features
                padding = np.zeros((X_processed.shape[0], expected_feature_count - X_processed.shape[1]))
                X_processed = np.hstack([X_processed, padding])

        # For hybrid model, reshape as needed
        X_reshaped = X_processed.reshape((X_processed.shape[0], X_processed.shape[1], 1))

        # Get predictions from hybrid model
        hybrid_pred = hybrid_model.predict(X_reshaped).flatten()

        # Get predictions from XGBoost and Random Forest
        xgb_pred = xgb_model.predict(X_processed)
        rf_pred = rf_model.predict(X_processed)

        # Format text for DistilBERT
        text_data = [
            f"Target City: {targetCity}, Current City: {currentCity}, "
            f"Start Date: {startDate}, Duration: {duration}, "
            f"Budget: {budget}, Phone Provided: {phone_provided}, "
            f"Distance: {distance}, Safety: {safety}, "
            f"Income: {income}, Lifestyle: {lifestyle}, "
            f"Pages Visited: {pages_visited}, Key Pages: {key_pages_visited}"
        ]

        # Tokenize text
        encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")

        # Get DistilBERT predictions
        with torch.no_grad():
            bert_output = bert_model(**encodings)
            bert_pred = bert_output.logits.numpy().flatten()

        # Combine predictions using ensemble weights
        ensemble_pred = (
            weights[0] * hybrid_pred +
            weights[1] * xgb_pred +
            weights[2] * rf_pred +
            weights[3] * bert_pred
        )

        # Round and determine quality
        score = round(float(ensemble_pred[0]), 2)

        if score > 70:
            quality = "High Quality Lead 🌟"
            color = "green"
        elif score > 40:
            quality = "Medium Quality Lead 🔍"
            color = "orange"
        else:
            quality = "Low Quality Lead ❗"
            color = "red"

        return f"Lead Score: {score}\nQuality: {quality}"

    except Exception as e:
        return f"Prediction Error: {str(e)}"

# Create Gradio Interface
def create_gradio_interface():
    # Create input components
    inputs = [
        gr.Dropdown(CATEGORICAL_OPTIONS['targetCity'], label="Target City"),
        gr.Dropdown(CATEGORICAL_OPTIONS['currentCity'], label="Current City"),
        gr.Dropdown(CATEGORICAL_OPTIONS['startDate'], label="Start Date"),
        gr.Dropdown(CATEGORICAL_OPTIONS['duration'], label="Duration"),
        gr.Dropdown(CATEGORICAL_OPTIONS['budget'], label="Budget"),
        gr.Dropdown(CATEGORICAL_OPTIONS['phone_provided'], label="Phone Provided"),
        gr.Dropdown(CATEGORICAL_OPTIONS['distance'], label="Distance"),
        gr.Dropdown(CATEGORICAL_OPTIONS['safety'], label="Safety"),
        gr.Dropdown(CATEGORICAL_OPTIONS['income'], label="Income"),
        gr.Dropdown(CATEGORICAL_OPTIONS['lifestyle'], label="Lifestyle"),
        gr.Number(label="Pages Visited", minimum=0, maximum=10),
        gr.Number(label="Key Pages Visited", minimum=0, maximum=10),
        gr.Number(label="Food Preferences", minimum=0, maximum=10),
        gr.Number(label="Transport Preferences", minimum=0, maximum=10),
        gr.Number(label="Accommodation Preferences", minimum=0, maximum=10),
        gr.Number(label="Key Pages Ratio", minimum=0, maximum=1),
        gr.Number(label="Budget Income Match", minimum=0, maximum=1),
        gr.Number(label="Is Local Travel", minimum=0, maximum=1)
    ]

    # Create Gradio Blocks for more flexibility
    with gr.Blocks() as demo:
        gr.Markdown("# 🚀 Lead Scoring Predictor")

        with gr.Row():
            with gr.Column():
                # Input components
                input_components = [gr.Dropdown(choices, label=label) if isinstance(choices, list) else gr.Number()
                                    for choices, label in zip(
                                        list(CATEGORICAL_OPTIONS.values()) + [None]*8,
                                        [k for k in CATEGORICAL_OPTIONS.keys()] +
                                        ["Pages Visited", "Key Pages Visited", "Food Preferences",
                                         "Transport Preferences", "Accommodation Preferences",
                                         "Key Pages Ratio", "Budget Income Match", "Is Local Travel"]
                                    )]

                # Predict button
                predict_btn = gr.Button("Predict Lead Score")

                # Random Generation button
                random_btn = gr.Button("Generate Random Lead")

            with gr.Column():
                # Output component
                output = gr.Textbox(label="Prediction Result")

        # Random generation event
        random_btn.click(
            fn=generate_random_lead,
            outputs=input_components
        )

        # Prediction event
        predict_btn.click(
            fn=predict_lead_score,
            inputs=input_components,
            outputs=output
        )

    return demo

# Main function
def main():
    interface = create_gradio_interface()
    interface.launch(share=True)

# Run the app
if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 18 arguments for function <function gradio_predict_lead_score at 0x7a90b2264540>, received 15.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 18 arguments for function <function gradio_predict_lead_score at 0x7a90b2264540>, received 15.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c54f79ac5a1fd1b88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c54f79ac5a1fd1b88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
from tensorflow.keras.models import load_model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import gradio as gr
import random

# ---------------------------
# Your existing functions
# ---------------------------
def load_and_prepare_models():
    # Load hybrid model
    hybrid_model = load_model('lead_scoring_hybrid.keras')

    # Load XGBoost model
    with open('xgb_model.pkl', 'rb') as f:
        xgb_model = pickle.load(f)

    # Load Random Forest model
    with open('rf_model.pkl', 'rb') as f:
        rf_model = pickle.load(f)

    # Load DistilBERT model and tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert_lead_scoring')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert_lead_scoring')

    # Load preprocessor
    with open('preprocessor.pkl', 'rb') as f:
        preprocessor = pickle.load(f)

    # Load ensemble weights
    with open('ensemble_weights.pkl', 'rb') as f:
        weights = pickle.load(f)

    return hybrid_model, xgb_model, rf_model, model, tokenizer, preprocessor, weights

def predict_lead_score(lead_data, models=None):
    """
    Predict lead score for a single lead record or a dataframe of leads

    Parameters:
    lead_data (dict or pd.DataFrame): Lead data to score
    models (tuple): Tuple of loaded models and preprocessing objects

    Returns:
    float or np.array: Predicted lead score(s)
    """
    if isinstance(lead_data, dict):
        lead_data = pd.DataFrame([lead_data])

    if models is None:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = load_and_prepare_models()
    else:
        hybrid_model, xgb_model, rf_model, bert_model, tokenizer, preprocessor, weights = models

    if 'preferences_specified' not in lead_data.columns:
        lead_data['preferences_specified'] = (
            (lead_data['food_preferences'] > 0).astype(int) +
            (lead_data['transport_preferences'] > 0).astype(int) +
            (lead_data['accommodation_preferences'] > 0).astype(int)
        )

    try:
        X_processed = preprocessor.transform(lead_data)
        xgb_feature_count = xgb_model.num_features()
        if X_processed.shape[1] != xgb_feature_count:
            print(f"Feature mismatch: preprocessor produced {X_processed.shape[1]} features, but XGBoost expects {xgb_feature_count}")
            X_processed_xgb = X_processed[:, :xgb_feature_count]
        else:
            X_processed_xgb = X_processed
        X_reshaped = X_processed.reshape((X_processed.shape[0], X_processed.shape[1], 1))
        hybrid_pred = hybrid_model.predict(X_reshaped).flatten()
        xgb_pred = xgb_model.predict(X_processed_xgb)
        rf_pred = rf_model.predict(X_processed_xgb)

        text_data = []
        for _, row in lead_data.iterrows():
            text = (
                f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
                f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
                f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
                f"Distance: {row['distance']}, Safety: {row['safety']}, "
                f"Income: {row['income']}, Lifestyle: {row['lifestyle']}, "
                f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}, "
                f"Food Preferences: {row['food_preferences']}, Transport: {row['transport_preferences']}, "
                f"Accommodation: {row['accommodation_preferences']}"
            )
            text_data.append(text)

        encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")
        with torch.no_grad():
            bert_output = bert_model(**encodings)
            bert_pred = bert_output.logits.numpy().flatten()

        ensemble_pred = (
            weights[0] * hybrid_pred +
            weights[1] * xgb_pred +
            weights[2] * rf_pred +
            weights[3] * bert_pred
        )

        if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
            return ensemble_pred[0]
        return ensemble_pred

    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        try:
            text_data = []
            for _, row in lead_data.iterrows():
                text = (
                    f"Target City: {row['targetCity']}, Current City: {row['currentCity']}, "
                    f"Start Date: {row['startDate']}, Duration: {row['duration']}, "
                    f"Budget: {row['budget']}, Phone Provided: {row['phone_provided']}, "
                    f"Pages Visited: {row['pages_visited']}, Key Pages: {row['key_pages_visited']}"
                )
                text_data.append(text)

            encodings = tokenizer(text_data, truncation=True, padding=True, max_length=128, return_tensors="pt")
            with torch.no_grad():
                bert_output = bert_model(**encodings)
                bert_pred = bert_output.logits.numpy().flatten()

            scaled_pred = bert_pred * 100
            if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
                return float(scaled_pred[0])
            return scaled_pred

        except Exception as fallback_error:
            print(f"Fallback prediction failed: {str(fallback_error)}")
            scores = []
            for _, row in lead_data.iterrows():
                score = 0
                if row['budget'] == 'High':
                    score += 30
                elif row['budget'] == 'Medium':
                    score += 20
                else:
                    score += 10
                if row['phone_provided'] == 'Yes':
                    score += 15
                score += min(row['pages_visited'] * 2, 20)
                score += min(row['key_pages_visited'] * 5, 25)
                scores.append(score)
            if isinstance(lead_data, pd.DataFrame) and len(lead_data) == 1:
                return scores[0]
            return np.array(scores)

# ---------------------------
# Gradio UI Integration
# ---------------------------
# Load models once for efficiency
models = load_and_prepare_models()

def predict_from_ui(
    targetCity, currentCity, startDate, duration, budget, phone_provided, distance,
    safety, income, lifestyle, pages_visited, key_pages_visited, food_preferences,
    transport_preferences, accommodation_preferences, key_pages_ratio, budget_income_match,
    is_local_travel
):
    # Prepare input as a dictionary
    input_data = {
        'targetCity': targetCity,
        'currentCity': currentCity,
        'startDate': startDate,
        'duration': duration,
        'budget': budget,
        'phone_provided': phone_provided,
        'distance': distance,
        'safety': safety,
        'income': income,
        'lifestyle': lifestyle,
        'pages_visited': pages_visited,
        'key_pages_visited': key_pages_visited,
        'food_preferences': food_preferences,
        'transport_preferences': transport_preferences,
        'accommodation_preferences': accommodation_preferences,
        'key_pages_ratio': key_pages_ratio,
        'budget_income_match': budget_income_match,
        'is_local_travel': is_local_travel
    }
    score = predict_lead_score(input_data, models)
    return score

def generate_random_input():
    # Generate random values for each field
    targetCity = random.choice(["Mumbai", "Pune", "Delhi", "Not specified"])
    currentCity = random.choice(["Bangalore", "Mumbai", "Chennai", "Not specified"])
    startDate = random.choice(["Within 30 days", "31-90 days", "Not specified"])
    duration = random.choice(["8-30 days", "1-7 days", "Not specified"])
    budget = random.choice(["High", "Medium", "Low"])
    phone_provided = random.choice(["Yes", "No"])
    distance = random.choice(["Long", "Short", "Not specified"])
    safety = random.choice(["High", "Medium", "Not specified"])
    income = random.choice(["High", "Medium", "Low"])
    lifestyle = random.choice(["Luxury", "Active", "Budget"])
    pages_visited = random.randint(0, 10)
    key_pages_visited = random.randint(0, 5)
    food_preferences = random.randint(0, 3)
    transport_preferences = random.randint(0, 3)
    accommodation_preferences = random.randint(0, 3)
    key_pages_ratio = round(random.uniform(0, 1), 2)
    budget_income_match = random.choice([0, 1])
    is_local_travel = random.choice([0, 1])

    return (targetCity, currentCity, startDate, duration, budget, phone_provided, distance,
            safety, income, lifestyle, pages_visited, key_pages_visited, food_preferences,
            transport_preferences, accommodation_preferences, key_pages_ratio, budget_income_match,
            is_local_travel)

# Define the Gradio interface layout
with gr.Blocks() as demo:
    gr.Markdown("## Lead Scoring Prediction UI")
    with gr.Row():
        with gr.Column():
            targetCity_input = gr.Textbox(label="Target City", value="Mumbai")
            currentCity_input = gr.Textbox(label="Current City", value="Bangalore")
            startDate_input = gr.Textbox(label="Start Date", value="Within 30 days")
            duration_input = gr.Textbox(label="Duration", value="8-30 days")
            budget_input = gr.Dropdown(label="Budget", choices=["High", "Medium", "Low"], value="High")
            phone_input = gr.Dropdown(label="Phone Provided", choices=["Yes", "No"], value="Yes")
            distance_input = gr.Textbox(label="Distance", value="Long")
            safety_input = gr.Textbox(label="Safety", value="High")
            income_input = gr.Textbox(label="Income", value="High")
            lifestyle_input = gr.Textbox(label="Lifestyle", value="Luxury")
        with gr.Column():
            pages_visited_input = gr.Slider(label="Pages Visited", minimum=0, maximum=10, step=1, value=6)
            key_pages_visited_input = gr.Slider(label="Key Pages Visited", minimum=0, maximum=5, step=1, value=3)
            food_preferences_input = gr.Slider(label="Food Preferences", minimum=0, maximum=3, step=1, value=2)
            transport_preferences_input = gr.Slider(label="Transport Preferences", minimum=0, maximum=3, step=1, value=1)
            accommodation_preferences_input = gr.Slider(label="Accommodation Preferences", minimum=0, maximum=3, step=1, value=1)
            key_pages_ratio_input = gr.Slider(label="Key Pages Ratio", minimum=0, maximum=1, step=0.01, value=0.5)
            budget_income_match_input = gr.Radio(label="Budget Income Match", choices=[0, 1], value=1)
            is_local_travel_input = gr.Radio(label="Is Local Travel", choices=[0, 1], value=0)

    predict_button = gr.Button("Predict Lead Score")
    random_button = gr.Button("Generate Random Input")
    output_text = gr.Textbox(label="Predicted Lead Score")

    predict_button.click(
        predict_from_ui,
        inputs=[targetCity_input, currentCity_input, startDate_input, duration_input, budget_input, phone_input,
                distance_input, safety_input, income_input, lifestyle_input, pages_visited_input, key_pages_visited_input,
                food_preferences_input, transport_preferences_input, accommodation_preferences_input, key_pages_ratio_input,
                budget_income_match_input, is_local_travel_input],
        outputs=output_text
    )

    random_button.click(
        generate_random_input,
        outputs=[targetCity_input, currentCity_input, startDate_input, duration_input, budget_input, phone_input,
                 distance_input, safety_input, income_input, lifestyle_input, pages_visited_input, key_pages_visited_input,
                 food_preferences_input, transport_preferences_input, accommodation_preferences_input, key_pages_ratio_input,
                 budget_income_match_input, is_local_travel_input]
    )

demo.launch()
